Try to merge in overloads from handcoded and other maintainers 
----
- clone to repos 
- new docstubs 
- merge in overloads




#### todo 
enrich  
- [ ] --no-docstring still copies over the class docstring in array.array
- [x] check if types for module constants are copied over 
   - [x] Create method to Copy TypeVars and TypeAliasses from source to target ( needed for future merges) 
        - [x] TypeVar
        - [x] TypeAlias
        - cmath has some incorrect type definitions for _C 
- [ ] Bug: Imports are copied even if they are inside an if statement (repos\rp2040\rp2\asm_pio.pyi :`from rp2 import PIOASMEmit` ) 
- [ ] ? create a method to forcefully copy over a class and all methods while merging 
        Q&D Workaround for mow with appending a piece of text.


- [x] avoid defining AnyReadableBuf and AnyWritableBuf in multile modules ( _mpy_shed module)
- [x] for now: Include in docstub MODULE_GLUE 
- [x] manual addition of some classes
- [x] rp2 and _rp2 modules need manual adjustments
    - [x] there are quite a few classes that are not/partly documented .rst files
    - [x] the several classes in `_rp2` are documented to be in `rp2` module
        - [x] `PIOASMEmit` class is missing in `rp2` module
        - may also need a way to remove a class from a module



 copy from C:\Users\josverl\.vscode\extensions\ms-python.vscode-pylance-2024.9.1\dist\typeshed-fallback\stdlib\cmath.pyi


```mermaid
flowchart TD
    A[Start] --> B[Clone Repos]
    B --> C[Create Docstubs]
    C --> D{MIT?}
    D -->|Yes| E[Use Howards Typeshed]
    D -->|No| F[Use IntelliJ Micropython Plugin]
    E --> G[Copy _mpy_shed to Reference Folder]
    F --> G[Copy _mpy_shed to Reference Folder]
    G --> H[Enrich from rp2040]
    H --> I[Sanity Check on Formatting]
    I --> J[Merge Redacted Typehints]
    J --> K[Sanity Check on Formatting]
    K --> L[Add _rp2 Module]
    L --> M[Copy _mpy_shed to Reference Folder]
    M --> N[Add rp2 Modules]
    N --> O[Fix u-module Imports]
    O --> P[Sanity Check on Formatting]
    P --> Q[Add Missing HASH Class]
    Q --> R{Clean rp2 Modules?}
    R -->|Yes| S[Clean rp2 Modules]
    R -->|No| T[Skip Cleaning]
    S --> U[Sanity Check on Formatting]
    T --> U[Sanity Check on Formatting]
    U --> V[Clean Formatting]
    V --> W[Run Autoflake]
    W --> X[Run Isort]
    X --> Y[Remove Duplicate Comments]
    Y --> Z[Final Sanity Check]
    Z --> AA[Run Pyright]
    AA --> AB[End]
```


In [1]:
# set parameters
from pathlib import Path


version = "v1.24.0"
flat_version = version.replace(".", "_")

# Destination
reference_p = Path("./scratch/micropython-reference")

# source of manual stubs ( _mpy_shed & rp2 )
manual_p = Path("repos\\micropython-stubs\\stubs\\_manual")
enrich_from_rp2040 = True
MIT = True


if MIT:
    # Howards Typeshed
    overload_typestubs = Path("repos\\PyBoardTypeshed\\micropython_typesheds")
else:
    # IntelliJ Micropython plugin
    overload_typestubs = Path("repos\\intellij-micropython\\typehints")

In [2]:
import shutil


# copy _mpy_shed to the reference folder
def copy_module(module: str | Path, src: Path = manual_p):
    target = reference_p / module
    if target.exists():
        shutil.rmtree(target)
    if (src / module).is_dir():
        shutil.copytree(src / module, target)
    else:
        shutil.copy(src / module, target)

In [3]:
# BLANK SLATE
! git -C scratch reset .
! git -C scratch clean . -f

# remove the old ref folder if it exists
if reference_p.exists():
    # remove the folder if it exists
    ! rmdir /s /q {reference_p}

# create docstubs without cleaning up the .rst  docstrings
! stubber docstubs --version {version} --stub-path scratch --no-autoflake 
# --no-clean-rst

! ren .\scratch\micropython-{flat_version}-docstubs micropython-reference

! black scratch
! git -C scratch add .

Unstaged changes after reset:
D	micropython-reference/_mpy_shed/__init__.pyi
D	micropython-reference/_mpy_shed/_collections_abc.pyi
D	micropython-reference/_mpy_shed/collections/__init__.pyi
D	micropython-reference/_mpy_shed/collections/abc.pyi
D	micropython-reference/_rp2/DMA.pyi
D	micropython-reference/_rp2/Flash.pyi
D	micropython-reference/_rp2/PIO.pyi
D	micropython-reference/_rp2/StateMachine.pyi
D	micropython-reference/_rp2/__init__.pyi
D	micropython-reference/_rp2/irq.pyi
D	micropython-reference/_thread/__init__.pyi
D	micropython-reference/array/__init__.pyi
D	micropython-reference/asyncio/__init__.pyi
D	micropython-reference/binascii/__init__.pyi
D	micropython-reference/bluetooth/__init__.pyi
D	micropython-reference/btree/__init__.pyi
D	micropython-reference/cmath/__init__.pyi
D	micropython-reference/collections/__init__.pyi
D	micropython-reference/cryptolib/__init__.pyi
D	micropython-reference/deflate/__init__.pyi
D	micropython-reference/errno/__init__.pyi
D	micropython-referen

Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\_thread\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\cmath\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\cryptolib\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\deflate\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\errno\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\esp\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\espnow\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\array\__init__.pyi
reformatted D:\mypython\micropython-stubber\scratch\micropython-reference\binascii\__init__.pyi
reformatted D:\mypython\mic

In [4]:
# ! git -C scratch reset .
# assume root of workspace is the current directory

if enrich_from_rp2040:
    # pull in josverl's handcoded rp2040 stubs
    !stubber enrich --params-only --stubs scratch --docstubs repos\rp2040


17:31:50 | ℹ️ micropython-stubber 1.23.2a0
17:31:50 | ℹ️ Enriching scratch with repos\rp2040
17:31:50 | ℹ️ Enrich folder scratch.
17:31:50 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\rp2\asm_pio.pyi
17:31:50 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\rp2\__init__.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\DMA.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\Flash.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\irq.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\PIO.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\PIOASMEmit.py
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\__init__.pyi from 
repos\rp2040\_rp2\StateMachine.pyi
17:31:51 | ℹ️ Merge scratch\micropython-reference\rp2\

In [5]:
### sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [6]:
## Merge redacted typehints from the overload source with docstubs
!stubber enrich --params-only --stubs scratch --docstubs {overload_typestubs}


17:32:05 | ℹ️ micropython-stubber 1.23.2a0
17:32:05 | ℹ️ Enriching scratch with repos\PyBoardTypeshed\micropython_typesheds
17:32:05 | ℹ️ Enrich folder scratch.
17:32:05 | ℹ️ Merge scratch\micropython-reference\_thread\__init__.pyi from 
repos\PyBoardTypeshed\micropython_typesheds\_thread.pyi
17:32:06 | ℹ️ Merge scratch\micropython-reference\array\__init__.pyi from 
repos\PyBoardTypeshed\micropython_typesheds\array.pyi
17:32:06 | ℹ️ Change __getitem__ to @overload
17:32:06 | ℹ️ Change __setitem__ to @overload
17:32:06 | ℹ️ Merge scratch\micropython-reference\asyncio\__init__.pyi from 
repos\PyBoardTypeshed\micropython_typesheds\uasyncio.pyi
17:32:07 | ℹ️ Merge scratch\micropython-reference\binascii\__init__.pyi from 
repos\PyBoardTypeshed\micropython_typesheds\binascii.pyi
17:32:07 | ℹ️ Merge scratch\micropython-reference\binascii\__init__.pyi from 
repos\PyBoardTypeshed\micropython_typesheds\ubinascii.pyi
17:32:07 | ℹ️ Merge scratch\micropython-reference\bluetooth\__init__.pyi from 
r

In [7]:
# sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [22]:
# add _rp2 module
! mkdir scratch\micropython-reference\_rp2
!copy .\repos\rp2040\_rp2\*.* .\scratch\micropython-reference\_rp2\*.*

A subdirectory or file scratch\micropython-reference\_rp2 already exists.


.\repos\rp2040\_rp2\DMA.pyi
.\repos\rp2040\_rp2\Flash.pyi
.\repos\rp2040\_rp2\irq.pyi
.\repos\rp2040\_rp2\PIO.pyi
.\repos\rp2040\_rp2\PIOASMEmit.py
.\repos\rp2040\_rp2\StateMachine.pyi
.\repos\rp2040\_rp2\__init__.pyi
        7 file(s) copied.


In [9]:
## Copy _mpy_shed to the reference folder
copy_module("_mpy_shed", manual_p)
copy_module("_thread", manual_p)

In [10]:
# Add rp2 modules ( may be an overkill?)
copy_module("_rp2", manual_p / "rpi")

In [11]:
# fix u-module imports and replace stdlib imports with _mpy_shed

from pathlib import Path


UMOD_TO_UMOD = [
    "from uarray import",
    "from uio import",
    "from usocket import",
    "from uos import",
    "from usys import",
    "from utime import",
]


NO_IMPORT = [
    "from io import AnyReadableBuf",
    "from io import AnyWritableBuf",
    "from uio import AnyReadableBuf",
    "from uio import AnyWritableBuf",
    "from io import StrOrBytesPath",
    "from uio import StrOrBytesPath",
]


REPLACE = [
    ("from os import AbstractBlockDev", "from _mpy_shed import AbstractBlockDev"),
    ("from io import IOBase", "from _mpy_shed import IOBase"),
    ("from uio import IOBase", "from _mpy_shed import IOBase"),
]


# Iterate over all files in the directory
# reference_p = Path("scratch/micropython-v1_23_0-docstubs")


for file in reference_p.rglob("*.p*"):
    # Check if the file is a regular file
    if file.is_file():
        # Open the file in read mode
        with file.open(mode="r", encoding="utf-8") as f:
            # Read the contents of the file
            lines = f.readlines()

        # Open the file in write mode
        with file.open(mode="w", encoding="utf-8") as f:
            # Iterate over each line in the file
            for line in lines:
                if "import" not in line:
                    f.write(line)
                    continue

                # Check if the line starts with
                if any(line.startswith(l) for l in UMOD_TO_UMOD):
                    # Modify the line to remove the `u` prefix
                    line = "from " + line[6:]

                # always replace the imports
                for old, new in REPLACE:
                    if line.startswith(old):
                        line = f"{new}\n"
                        break

                if any(line.startswith(l) for l in NO_IMPORT):
                    # remove the line
                    # line = "# " + line
                    line = ""

                # Write the modified line to the file
                f.write(line)

In [12]:
# sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 

All checks passed!


In [13]:
# Add missing HASH class to hashlib


def append_partial_module(module: str):
    with open(manual_p / "partial_modules" / f"{module}.pyi") as f:
        content = f.read()
    with open(reference_p / module / "__init__.pyi", "a") as f:
        f.write(content)


append_partial_module("hashlib")
append_partial_module("random")

In [14]:
# clean / change rp2 / _rp2

# # if not MIT:
# # Append the class PIOASMEmit to the rp2 module
# append_partial_module("rp2")


clean_rp2_modules = False
if clean_rp2_modules:

    # remove the submodules  to avoid duplication in multiple files

    for name in ["DMA", "FLASH", "PIO", "StateMachine"]:
        submodule = reference_p / f"rp2/{name}.pyi"
        submodule.unlink(missing_ok=True)

    # remove the relative imports from the rp2 module
    rp2 = Path(reference_p / "rp2" / "__init__.pyi")

    with rp2.open(mode="r", encoding="utf-8") as f:
        # Read the contents of the file
        lines = f.readlines()

    # Open the file in write mode

    # with rp2.open(mode="w", encoding="utf-8") as f:
    #     # Iterate over each line in the file
    #     for line in lines:
    #         # Check if the line starts with
    #         if line.startswith("from .") or line == "from rp2 import PIOASMEmit":
    #             line = "# " + line

    #         # Write the modified line to the file
    #         f.write(line)

In [15]:
# sanity check on formatting
! ruff check scratch\micropython-reference --ignore F4 --ignore UP0 --ignore F811 


All checks passed!


In [ ]:
# Clean formatting and upgrade typing notation to 3.10
#  except :
# - non-pep585-annotation (UP006)  - Keep from typing import List to support Python 3.7 , 3.8 
 
! ruff check {reference_p} --fix --unsafe-fixes --ignore UP006 --ignore  UP035 --ignore F405



All checks passed!


In [17]:
# run autoflake on all files - one by one to get the __init__.pyi files
for f in reference_p.rglob("*.pyi"):
    ! autoflake {f} --in --imports typing_extensions,_mpy_shed,_typeshed


In [ ]:
# clean up the formatting and import orders by running isort
! isort {reference_p}

# remove duplicate comments created by merging multiple files 

def remove_duplicate_comments(file):
    with open(file, encoding="utf-8") as f:
        lines = f.readlines()

    comments = {}
    with open(file, "w", encoding="utf-8") as f:
        for line in lines:
            if line.startswith("# MCU:") or line.startswith("# Stubber"):
                if line in comments:
                    continue
                comments[line] = True
            f.write(line)


for file in reference_p.rglob("*.pyi"):
    remove_duplicate_comments(file)

Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\uio.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\usocket.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\array\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\asyncio\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\bluetooth\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\btree\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\cmath\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\collections\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\cryptolib\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\deflate\__init__.pyi
Fixing D:\mypython\micropython-stubber\scratch\micropython-reference\errno\__init__.pyi
Fixing D:\mypython\micropython-s

In [23]:
print(f"Reference baked from Source {overload_typestubs}\n{'='*40} ")
!cd "d:\mypython\micropython-stubber\scratch\micropython-reference" && pyright .
!cd "d:\mypython\micropython-stubber"


Reference baked from Source repos\PyBoardTypeshed\micropython_typesheds
Please install the new version or set PYRIGHT_PYTHON_FORCE_VERSION to `latest`

d:\mypython\micropython-stubber\scratch\micropython-reference\espnow\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-reference\espnow\__init__.pyi:468:35 - information: "value" is not defined (reportUndefinedVariable)
d:\mypython\micropython-stubber\scratch\micropython-reference\io\__init__.pyi
  d:\mypython\micropython-stubber\scratch\micropython-reference\io\__init__.pyi:208:44 - information: "TextIOWrapper" is not defined (reportUndefinedVariable)
  d:\mypython\micropython-stubber\scratch\micropython-reference\io\__init__.pyi:216:33 - information: "_OpenTextModeWriting" is not defined (reportUndefinedVariable)
  d:\mypython\micropython-stubber\scratch\micropython-reference\io\__init__.pyi:216:78 - information: "TextIOWrapper" is not defined (reportUndefinedVariable)
  d:\mypython\micropython-stubber\scratch\micropy